In [2]:
import pickle
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch
import youtokentome as yttm
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [3]:
device = torch.device("cuda:0")
cpu = torch.device("cpu")

In [4]:
!nvidia-smi

Thu Nov 21 11:33:50 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Load data 

In [5]:
story_path = "corpus/story_data_punct_del_em.pkl"
with open(story_path, 'rb') as f:
    data = pickle.load(f)

In [6]:
STORY_VAL = 34000
data_batch = data[:STORY_VAL]

In [7]:
def samples2text(data, story_size=300):
    new_data = []
    for i in range(len(data)):
        text = " ".join(data[i]).replace(" .",".").replace(" ,", ",").replace(" ?", "?")
        
        if len(text) <= story_size:
            new_data.append(text)
    
    return new_data

In [8]:
data_batch = list(samples2text(data_batch))

In [9]:
random_story = np.random.randint(0, len(data_batch)-1)
data_batch[random_story]

'Привет, Позор ! Сегодня захавал 2 пачки дошика, пес поел тушенки из фикс прайса. Щас лежим в одной комнате, пропердели все так, что если зажечь зажигалку, то воздух по всей комнате воспламенится. Мне страшно.'

## Split data

In [11]:
from data_tools import split_data, get_bpe_tokenizer, get_unknown_ngrams

In [11]:
train_texts, test_texts = split_data(data_batch, train_size=0.75)

Total samples: 21061

Traning size: 15795 | Validating size: 5266


## Apply BPE 

In [12]:
train_txt = 'train_bpe.txt'
bpe_model_name = "story_bpe.yttm"
BPE_VOCAB_SIZE = 1000

tokenizer = get_bpe_tokenizer(train_texts, train_txt_path=train_txt, bpe_model_name=bpe_model_name,
                              vocab_size=BPE_VOCAB_SIZE)

In [13]:
random_id = np.random.randint(1, len(train_texts)-1)

print(train_texts[random_id])
print("")
print(*tokenizer.encode(train_texts[random_id]))

К словам физика Вот например, пойдёшь в 18 лет в армию, на войну, поубиваешь там всех, а потом в магазин придешь за водкой и тебе скажут мальчик тебе сколько лет? Водка с 21..

215 877 17 911 8 163 227 207 152 19 172 830 20 753 18 37 321 144 249 66 328 144 982 281 404 152 144 240 191 20 147 16 394 187 487 395 330 724 195 450 144 372 289 581 10 245 255 321 175 277 18 346 145 547 7 141 163 366 9 151 350 304 14 547 7 883 464 328 50 227 5 18 163 141 283 39 258


In [14]:
train_token_ids = tokenizer.encode(train_texts, bos=True, eos=True)
test_token_ids = tokenizer.encode(test_texts, bos=True, eos=True)

In [15]:
get_unknown_ngrams(test_token_ids)

Unknown n-grams in validation set:  0


## Create data-loaders

In [16]:
from torch.nn.utils.rnn import pad_sequence

In [17]:
def get_loaders(data, padding_value=0, batch_size=512, shuffle=True):
    input_seq = []
    target_seq = []
    
    for story in data:
        input_seq.append(torch.tensor(story[:-1]))
        target_seq.append(torch.tensor(story[1:]))
    
    input_seq = pad_sequence(input_seq, batch_first=True, padding_value=padding_value)
    target_seq = pad_sequence(target_seq, batch_first=True, padding_value=padding_value)

    data = torch.utils.data.TensorDataset(input_seq, target_seq)
    data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=shuffle)
    
    return data_loader

In [18]:
train_loader = get_loaders(train_token_ids)
test_loader = get_loaders(test_token_ids, shuffle=True)

## Init Language Model 

In [14]:
from model_tools import dependency_mask, positional_encoding

In [20]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, backbone, emb_dropout=0.0):
        super().__init__()
        self.embedding_size = embedding_size
        self.embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.emb_dropout = nn.Dropout(emb_dropout)
        self.backbone = backbone
        self.out = nn.Linear(embedding_size, vocab_size)
    
    def forward(self, seed_token_ids):

        batch_size, max_in_length = seed_token_ids.shape

        seed_padding_mask = seed_token_ids == 0
        dep_mask = dependency_mask(max_in_length).to(seed_token_ids.device)
        
        seed_embs = self.embeddings(seed_token_ids)  
        pos_codes = positional_encoding(max_in_length,
                                             self.embedding_size).unsqueeze(0).to(seed_embs.device)
        seed_embs = seed_embs + pos_codes
        seed_embs = self.emb_dropout(seed_embs)

        
        target_features = seed_embs
        target_features = self.backbone(seed_embs,
                                        mask=dep_mask,
                                        src_key_padding_mask=seed_padding_mask)
        
        logits = self.out(target_features)  
        return logits

In [21]:
class Transformer(nn.Module):
    def __init__(self):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=256, nhead=16, 
                                                        dim_feedforward=512, dropout=0.3)
        
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=3)
        self.init_weights()
        
    def forward(self, src, *args, **kwargs):
        # read dep_mask and src_key_padding_mask
        # Transpose seq to Batch First
        
        src = src.transpose(0, 1).contiguous()
        
        results = self.encoder(src, *args, **kwargs)
        
        results = results.transpose(0, 1).contiguous()
        
        return results

    def init_weights(self):
        for param in self.encoder.parameters():
            if param.dim() > 1:
                nn.init.xavier_uniform_(param)

In [22]:
vocab_size = tokenizer.vocab_size()
embedding_size = 256

model = LanguageModel(vocab_size, embedding_size, Transformer(), emb_dropout=0.1)
print('Params:', sum(t.numel() for t in model.parameters()))

Params: 2094312


In [23]:
model

LanguageModel(
  (embeddings): Embedding(1000, 256, padding_idx=0)
  (emb_dropout): Dropout(p=0.1, inplace=False)
  (backbone): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): Linear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=512, bias=True)
          (dropout): Dropout(p=0.3, inplace=False)
          (linear2): Linear(in_features=512, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.3, inplace=False)
          (dropout2): Dropout(p=0.3, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): Linear(in_features=256, out_features=256, bias=True)
         

In [24]:
LR = 2e-3
EPOCH = 40
reg_alpha = 0

In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=reg_alpha)
model = model.to(device)

## Train loop

In [19]:
from train_tools import train_loop

In [61]:
model = train_loop(model, device, optimizer, train_loader, test_loader, epoch_value=10, plot_loss=False)

Time: 0m 30s | Epoch: 1 / 10 | T-Loss: 3.057 | Val-Loss: 2.772
Time: 1m 0s | Epoch: 2 / 10 | T-Loss: 3.054 | Val-Loss: 2.774
Time: 1m 31s | Epoch: 3 / 10 | T-Loss: 3.049 | Val-Loss: 2.770
Time: 2m 1s | Epoch: 4 / 10 | T-Loss: 3.041 | Val-Loss: 2.760
Time: 2m 32s | Epoch: 5 / 10 | T-Loss: 3.037 | Val-Loss: 2.755
Time: 3m 2s | Epoch: 6 / 10 | T-Loss: 3.034 | Val-Loss: 2.752
Time: 3m 33s | Epoch: 7 / 10 | T-Loss: 3.030 | Val-Loss: 2.743
Time: 4m 4s | Epoch: 8 / 10 | T-Loss: 3.024 | Val-Loss: 2.734
Time: 4m 34s | Epoch: 9 / 10 | T-Loss: 3.020 | Val-Loss: 2.733
Time: 5m 5s | Epoch: 10 / 10 | T-Loss: 3.015 | Val-Loss: 2.732


In [ ]:
# Total Epoch: 60

## Eval

In [20]:
from eval_tools import create_greedy_text, BeamGenerator

In [71]:
create_greedy_text(model, tokenizer, "Я пошел в магазин утром", 40)

'Я пошел в магазин утром в магазине. В итоге все слышу какие то мужики и вижу как мужик смотрит нам, что он говорит, что это может это????'

In [73]:
beam_generator = BeamGenerator(model, tokenizer)

In [75]:
beam_gen_variants = beam_generator('Я пошел в магазин утром',100, beamsize=5, return_hypotheses_n=5)

for score, pred_txt in beam_gen_variants:
    print('--------------------')
    print(score)
    print(pred_txt)
    print()

--------------------
3.934856298521477
Я пошел в магазин утром был в магазин. Так вот, сегодня спросил, что он ответил, что на грани фантастики.<EOS>

--------------------
4.0701849869727855
Я пошел в магазин утром был в магазин. Так вот, сегодня спросил, что он ответил что на грани фантастики.<EOS>

--------------------
4.201108203673812
Я пошел в магазин утром был в магазин. Так вот, сегодня спросил, что он ответил, что на грани фантастики<EOS>

--------------------
4.324112428351587
Я пошел в магазин утром был в магазин. Так вот, сегодня спросил, что он ответил, что на грани фантастики !<EOS>

--------------------
4.398650195709798
Я пошел в магазин утром был в магазин. Так вот, сегодня спросил, что он ответил что на грани фантастики<EOS>



## Save PyTorch model

In [58]:
torch.save({
            'epoch': EPOCH,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': 3.203}, 'story_model_trs.pth')

## Load Tokenizer and PyTorch model(for Train)

In [ ]:
bpe_model_name = "story_bpe.yttm"
tokenizer = yttm.BPE(bpe_model_name)

In [ ]:
vocab_size = tokenizer.vocab_size()
embedding_size = 256

model = LanguageModel(vocab_size, embedding_size, Transformer(), emb_dropout=0.1)

LR = 2e-3
EPOCH = 40
reg_alpha = 0
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=reg_alpha)

checkpoint = torch.load(story_model_fldct.pth)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']